-sandbox
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://cdn2.hubspot.net/hubfs/438089/docs/training/dblearning-banner.png" alt="Databricks Learning" width="555" height="64">
</div>

-sandbox
&copy; 2018 Databricks, Inc. All rights reserved.<br/>

### Challenges
* Larger Data
* Faster data and decisions - seconds, minutes, hours not days or weeks after it is created
* Streaming Pipelines can be hard
* Realtime Dashboards and alerts - for the holiday season, promotional campaigns, track falling or rising trends

### Azure Databricks Solutions
* Connect Azure Databricks with a click of a button
* Easy streaming pipelines almost the same as batch - SQL, Python, Scala, Java & R
* Make this data avialable on Storage or ADL to end users in minutes not days or weeks. 

### Why Contoso Needs Streaming
* Sales up or down (rolling 24 hours, 1 hour), to identify trends that are good or bad
* Holidays and promotions - how are the performing in real time

-sandbox
## What is Structured Streaming?

<div style="width: 100%">
  <div style="margin: auto; width: 800px">
    <img src="http://spark.apache.org/docs/latest/img/structured-streaming-stream-as-a-table.png"/>
  </div>
</div>

Data is appended to the Input Table every _trigger interval_. For instance, if the trigger interval is 1 second, then new data is appended to the Input Table every seconds. (The trigger interval is analogous to the _batch interval_ in the legacy RDD-based Streaming API.)

####Azure Databricks for Streaming Analytics, Alerts, ETL & Data Engineers 

![arch](https://kpistoropen.blob.core.windows.net/collateral/roadshow/azure_roadshow_sde.png)

##![Spark Logo Tiny](https://kpistoropen.blob.core.windows.net/collateral/roadshow/logo_spark_tiny.png) *Part-1:* Create Streaming DataFrame

In [7]:
#schema for our streaming DataFrame

from pyspark.sql.types import *
schema = StructType([ \
  StructField("orderUUID", StringType(), True), \
  StructField("productId", IntegerType(), True), \
  StructField("userId", IntegerType(), True), \
  StructField("quantity", IntegerType(), True), \
  StructField("discount", DoubleType(), True), \
  StructField("orderTimestamp", TimestampType(), True)])

In [8]:
%fs ls /mnt/databricks-workshop-datasets/Contoso-retail/initech/streaming/orders/data

path,name,size
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/initech/streaming/orders/data/_spark_metadata/,_spark_metadata/,0
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/initech/streaming/orders/data/part-00000-0157c68f-e6b9-461a-bbe5-1de8f5bf7cae-c000.csv,part-00000-0157c68f-e6b9-461a-bbe5-1de8f5bf7cae-c000.csv,0
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/initech/streaming/orders/data/part-00000-07d59b0d-9a77-4789-a010-8c0c3cbb7ceb-c000.csv,part-00000-07d59b0d-9a77-4789-a010-8c0c3cbb7ceb-c000.csv,5204
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/initech/streaming/orders/data/part-00000-0e25b756-08c9-4d47-81e9-7fbb90646353-c000.csv,part-00000-0e25b756-08c9-4d47-81e9-7fbb90646353-c000.csv,0
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/initech/streaming/orders/data/part-00000-1814c789-3471-4e77-be90-20017c9643e6-c000.csv,part-00000-1814c789-3471-4e77-be90-20017c9643e6-c000.csv,0
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/initech/streaming/orders/data/part-00000-1d341d34-f86a-4d1a-895f-f7802ce37623-c000.csv,part-00000-1d341d34-f86a-4d1a-895f-f7802ce37623-c000.csv,5199
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/initech/streaming/orders/data/part-00000-29184294-9487-4c30-8178-5ad0e0b73cec-c000.csv,part-00000-29184294-9487-4c30-8178-5ad0e0b73cec-c000.csv,0
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/initech/streaming/orders/data/part-00000-335db11d-64af-4b60-abbf-f807b775bd5b-c000.csv,part-00000-335db11d-64af-4b60-abbf-f807b775bd5b-c000.csv,5205
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/initech/streaming/orders/data/part-00000-5d57ffdc-a4fa-4003-a4b6-04d8cda82052-c000.csv,part-00000-5d57ffdc-a4fa-4003-a4b6-04d8cda82052-c000.csv,5205
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/initech/streaming/orders/data/part-00000-665c87c6-e29e-406e-91c5-6b7eefe2c0ef-c000.csv,part-00000-665c87c6-e29e-406e-91c5-6b7eefe2c0ef-c000.csv,5200


In [9]:
#streaming DataFrame reader for data on Azure Storage

streaming_df = spark.readStream \
    .schema(schema) \
    .option("maxFilesPerTrigger", 1) \
    .csv("dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/initech/streaming/orders/data/part-*")

## What can we do with this Streaming DataFrame?

If you run the following cell, you'll get a continuously updating display of the number of records read from the stream so far. Note that we're just calling `display()` on our DataFrame, _exactly_ as if it were a DataFrame reading from a static data source.

To stop the continuous update, just cancel the query.

In [11]:
display(streaming_df)

orderUUID,productId,userId,quantity,discount,orderTimestamp
27cabbc2-9fc4-4e1f-b4ee-f52df4c5a6b6,7761,12316,1,0.0,2018-02-27T20:03:13.698+0000
1d34f144-bb13-466f-aeb5-977c8a667ad6,7750,19720,1,0.0,2018-02-27T20:03:19.152+0000
2a7adacf-8123-48b5-accf-43f844318918,7731,22573,5,0.0,2018-02-27T20:03:20.574+0000
70d91e90-7a2b-4c8d-909f-874f9d4164e2,7742,13189,4,0.0,2018-02-27T20:03:22.047+0000
f4b329af-bc8d-41f6-9ac0-687500a0fca2,7725,45432,5,0.0,2018-02-27T20:03:22.491+0000
b226e455-9ed1-48f7-bcda-4579f24150ff,7730,75608,1,0.1,2018-02-27T20:03:22.836+0000
c4650793-51a7-4f67-991b-14e83f7c1651,7757,46704,1,0.0,2018-02-27T20:03:25.409+0000
7de53ec1-028a-4d5f-aeca-c6223ac0d499,7761,35203,3,0.0,2018-02-27T20:03:26.541+0000
841bb8c9-e5d3-43e6-ba6d-2fa7428fa05f,7734,16943,1,0.0,2018-02-27T20:03:44.904+0000
47d79f60-1e47-4d60-b408-0a4367f92598,7727,68217,1,0.0,2018-02-27T20:03:46.450+0000


##![Spark Logo Tiny](https://kpistoropen.blob.core.windows.net/collateral/roadshow/logo_spark_tiny.png) *Part-2:* Transform Streaming DataFrame

### It's just a DataFrame

We can use normal DataFrame transformations on our streaming DataFrame. For example, let's group the number of orders by productId

<img src="https://spark.apache.org/docs/latest/img/structured-streaming-example-model.png"/>

In [14]:
from pyspark.sql.functions import *
top_products = streaming_df.groupBy("productId").agg(sum(col("quantity")).alias("total_units_by_product")).orderBy(desc("total_units_by_product"))

* Call `display` on `top_products`
* Turn the streaming table into a streaming bar chart

In [16]:
display(top_products)

productId,total_units_by_product
7742,12
7761,10
7725,8
7735,8
7750,5
7752,5
7731,5
7745,4
7726,4
7749,3


##![Spark Logo Tiny](https://kpistoropen.blob.core.windows.net/collateral/roadshow/logo_spark_tiny.png) *Part-3:* Streaming Joins

### Streaming Joins

Grouping by unkown product IDs is not that that exciting. Let's join the stream with the product lookup data set
* Use the join key productId
* Hint: Since both DataFrames have the same column name `productId`
* Use the duplicated columns trick documented here: https://docs.azuredatabricks.net/spark/latest/faq/join-two-dataframes-duplicated-column.html

Load the product lookup data from Azure Storage

In [20]:
product_lookup = spark.read.parquet("/mnt/databricks-workshop-datasets/Contoso-retail/initech/productsFull/")

Join the `streaming_df` with `product_lookup` on `productId`
* Hint: https://docs.azuredatabricks.net/spark/latest/faq/join-two-dataframes-duplicated-column.html

In [22]:
joined_df = streaming_df.join(product_lookup, "ProductID")

In [23]:
display(joined_df)

productId,orderUUID,userId,quantity,discount,orderTimestamp,Name,ProductNumber,Color,StandardCost,ListPrice,Size,Weight,ProductCategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,ThumbNailPhoto,ThumbnailPhotoFileName,ModifiedDate
7725,2eba0958-464e-480c-a928-ac8f5df9842e,47971,1,0.0,2018-02-27T20:05:16.024+0000,"Microsoft Surface Pro – 12.3""""""",L-0002-YE,yellow,1299.99,1299.99,"12.3""",null,2,2,2018-02-06T00:00:00.000+0000,2018-02-06T00:00:00.000+0000,null,UjBsR09EbGhVQUF4QVBjQUFBQUFBSUFBQUFDQUFJQ0FBQUFBZ0lBQWdBQ0FnSUNBZ01EQXdQOEFBQUQvQVAvL0FBQUEvLzhBL3dELy8vLy8vd0FBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQU13QUFaZ0FBbVFBQXpBQUEvd0F6QUFBek13QXpaZ0F6bVFBenpBQXovd0JtQUFCbU13Qm1aZ0JtbVFCbXpBQm0vd0NaQUFDWk13Q1paZ0NabVFDWnpBQ1ovd0RNQUFETU13RE1aZ0RNbVFETXpBRE0vd0QvQUFEL013RC9aZ0QvbVFEL3pBRC8vek1BQURNQU16TUFaak1BbVRNQXpETUEvek16QURNek16TXpaak16bVRNenpETXovek5tQURObU16Tm1aak5tbVRObXpETm0vek9aQURPWk16T1paak9abVRPWnpET1ovelBNQURQTU16UE1aalBNbVRQTXpEUE0velAvQURQL016UC9aalAvbVRQL3pEUC8vMllBQUdZQU0yWUFabVlBbVdZQXpHWUEvMll6QUdZek0yWXpabVl6bVdZenpHWXovMlptQUdabU0yWm1abVptbVdabXpHWm0vMmFaQUdhWk0yYVpabWFabVdhWnpHYVovMmJNQUdiTU0yYk1abWJNbVdiTXpHYk0vMmIvQUdiL00yYi9abWIvbVdiL3pHYi8vNWtBQUprQU01a0FacGtBbVprQXpKa0EvNWt6QUprek01a3pacGt6bVprenpKa3ovNWxtQUpsbU01bG1acGxtbVpsbXpKbG0vNW1aQUptWk01bVpacG1abVptWnpKbVovNW5NQUpuTU01bk1acG5NbVpuTXpKbk0vNW4vQUpuL001bi9acG4vbVpuL3pKbi8vOHdBQU13QU04d0Fac3dBbWN3QXpNd0EvOHd6QU13ek04d3pac3d6bWN3enpNd3ovOHhtQU14bU04eG1ac3htbWN4bXpNeG0vOHlaQU15Wk04eVpac3labWN5WnpNeVovOHpNQU16TU04ek1ac3pNbWN6TXpNek0vOHovQU16L004ei9ac3ovbWN6L3pNei8vLzhBQVA4QU0vOEFadjhBbWY4QXpQOEEvLzh6QVA4ek0vOHpadjh6bWY4enpQOHovLzltQVA5bU0vOW1adjltbWY5bXpQOW0vLytaQVArWk0vK1paditabWYrWnpQK1ovLy9NQVAvTU0vL01adi9NbWYvTXpQL00vLy8vQVAvL00vLy9adi8vbWYvL3pQLy8veUg1QkFFQUFCQUFMQUFBQUFCUUFERUFBQWovQVA4SkhFaXdvTUdEQ0JNcVhNaXdvY09IRUNOS25FaXhvc1dMR0ROcTNNaXhvOGVQSUVPS0hFbXlwTW1US0ZPcVhKa1JCWXFCTGhmR1pQblE1Y3QvTXhQbXBNblFwc0NaTm0vQ2ZCblRaODZnUTNIZVJNb1JhZEdsUXBVcUpmb1VaOUtuVkg5R3hWaFVLdENvVmFXS25aclZLOVNtVk1QdVZIdldyRmlzUGpkK0xidVc3dG12Yjh0Nm5KdVhJRnV0ZmJIMmxTdDA3dGEvZWVPeTNjbFRZdUd0alM4eWpVeTVzdVhMbURISGRSaldJR1BHSWpkREJBM1lMMlNRVlkrbXZRc1ZMMTZ5cUxPcWZ1eVd0bEhaYlR2K25ZMTc2RzY3SDM4RFRzMDY4R3JTa29NU04rNjIrZktRcXJXMlhlNmFlbTdDU2FmNmZxN2NlZXZUbWNPTEVoOVB2cno1OCtqVHExL1B2cjM3OSs4REFnQTc=,no image available,2018-02-06T00:00:00.000+0000
7725,95d37730-e385-4cf9-8645-50b94dd4cf59,70725,2,0.0,2018-02-27T20:03:59.797+0000,"Microsoft Surface Pro – 12.3""""""",L-0002-YE,yellow,1299.99,1299.99,"12.3""",null,2,2,2018-02-06T00:00:00.000+0000,2018-02-06T00:00:00.000+0000,null,UjBsR09EbGhVQUF4QVBjQUFBQUFBSUFBQUFDQUFJQ0FBQUFBZ0lBQWdBQ0FnSUNBZ01EQXdQOEFBQUQvQVAvL0FBQUEvLzhBL3dELy8vLy8vd0FBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQU13QUFaZ0FBbVFBQXpBQUEvd0F6QUFBek13QXpaZ0F6bVFBenpBQXovd0JtQUFCbU13Qm1aZ0JtbVFCbXpBQm0vd0NaQUFDWk13Q1paZ0NabVFDWnpBQ1ovd0RNQUFETU13RE1aZ0RNbVFETXpBRE0vd0QvQUFEL013RC9aZ0QvbVFEL3pBRC8vek1BQURNQU16TUFaak1BbVRNQXpETUEvek16QURNek16TXpaak16bVRNenpETXovek5tQURObU16Tm1aak5tbVRObXpETm0vek9aQURPWk16T1paak9abVRPWnpET1ovelBNQURQTU16UE1aalBNbVRQTXpEUE0velAvQURQL016UC9aalAvbVRQL3pEUC8vMllBQUdZQU0yWUFabVlBbVdZQXpHWUEvMll6QUdZek0yWXpabVl6bVdZenpHWXovMlptQUdabU0yWm1abVptbVdabXpHWm0vMmFaQUdhWk0yYVpabWFabVdhWnpHYVovMmJNQUdiTU0yYk1abWJNbVdiTXpHYk0vMmIvQUdiL00yYi9abWIvbVdiL3pHYi8vNWtBQUprQU01a0FacGtBbVprQXpKa0EvNWt6QUprek01a3pacGt6bVprenpKa3ovNWxtQUpsbU01bG1acGxtbVpsbXpKbG0vNW1aQUptWk01bVpacG1abVptWnpKbVovNW5NQUpuTU01bk1acG5NbVpuTXpKbk0vNW4vQUpuL001bi9acG4vbVpuL3pKbi8vOHdBQU13QU04d0Fac3dBbWN3QXpNd0EvOHd6QU13ek04d3pac3d6bWN3enpNd3ovOHhtQU14bU04eG1ac3htbWN4bXpNeG0vOHlaQU15Wk04eVpac3labWN5WnpNeVovOHpNQU16TU04ek1ac3pNbWN6TXpNek0vOHovQU16L004ei9ac3ovbWN6L3pNei8vLzhBQVA4QU0vOEFadjhBbWY4QXpQOEEvLzh6QVA4ek0vOHpadjh6bWY4enpQOHovLzltQVA5bU0vOW1adjltbWY5bXpQOW0vLytaQVArWk0vK1paditabWYrWnpQ

##![Spark Logo Tiny](https://kpistoropen.blob.core.windows.net/collateral/roadshow/logo_spark_tiny.png) *Part-4:* Calculate a Streaming Dashboard - Revenue by Product Name

### Calculate the Total Revenue by Product Name

* Now that we have the product `Name` let's use that instead of the `productId` to `groupBy`
* Also let's calculate the total revenue instead of just units sold
  * Use the `quanity` column and the `StandardCost` column

In [26]:
top_products = joined_df.groupBy("Name").agg(sum(col("quantity")*col("StandardCost")).alias("total_revenue_by_product")).orderBy(desc("total_revenue_by_product"))

In [27]:
display(top_products)

Name,total_revenue_by_product
Microsoft Surface Pro – 12.3”,32399.879999999997
"""Apple MacBook Pro - 15"" Display""""pple Mac""",21279.92
"Microsoft Surface Pro – 12.3""""""",19499.85
"""Samsung Notebook 9 Pro 13.3"" Touch-Screen Laptop""",13199.880000000001
"""Apple MacBook Pro - 15.4"" Display""""pple Mac""",9999.95
"Microsoft Surface Book 2 - 13.5""""""",7499.95
"""HP Spectre x360 2-in-1 13.3"" 4K Ultra HD Touch-Sc",5999.96
"""Google Pixelbook 12.3"" Touchscreen Chromebook""""oo",5099.97
"""HP Pavilion x360 2-in-1 14"" Touch-Screen Laptop""""",4999.950000000001
"""Samsung Notebook 9 spin 13.3"" Touch-Screen Laptop",4799.96


## Next Step

[Recommendations using ALS]($../4-ML/4-01 Recommendations using ALS)

&copy; 2018 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>